# Opgave adapter pattern
## Doel
- Oefenen met het adapter pattern

## Inleiding
Deze opgave gaat over gaat over een weerstation, ontwikkeld door het fictieve bedrijf ABCMega-Weather corp.

Het bedrijf heeft een weerstation ontwikkeld. Het weerstation heeft een touchscreen, waarmee een zeer uitgebreide grafische gebruikersinterface kan worden bediend. Het besturingsysteem is geschreven in Java.

Het eerste deel van de opgave gaat over de interactie tussen touchscreen en gebruikersinterface op het moment dat een gebruiker het scherm aanraakt. Omwille van eenvoud zijn andere zaken achterwege gelaten. In werkelijkheid is zo'n systeem veel complexer. Ook is omwille van de eenvoud de uitvoer niet grafisch maar op de console.

## Opzet van het besturingssysteem

In de package *ui* bevinden zich klassen van de gebruikersinterface.

De kern van de grafische gebruikersinterface is geïmplementeerd in de klasse **WeatherStationUserInterface**.

De communicatie met de hardware is gescheiden van de gebruikersinterface. De drivers voor verschillende hardware-componenten bevinden zich in de package *drivers*. In de package *drivers* bevinden zich packages voor verschillende categorieën hardware.

In de klasse Main bevindt zich de main-methode. Deze bevindt zich niet in de genoemde packages.

Maak de klasse *Main* met main-methode.

## De gebruikersinterface

Omwille van flexibiliteit implementeert de gebruikersinterface-klasse de interface **UserInterface**:
```Java
public interface UserInterface {

    void linkTouchScreenInputDevice(TouchScreenInputDevice touchScreenInputDevice);
    void onTouch(int x, int y);
    void onLongTouch(int x, int y);

}
```

In een werkelijke situatie zou deze interface veel uitgebreider zijn maar in deze opgave blijft het beperkt tot drie methoden.

Door middel van de methoden *onTouch* en *onLongTouch* wordt aan de gebruikersinterface doorgegeven dat de gebruiker op het touchscreen heeft gedrukt. Dit kan zowel kort als lang zijn. Deze methodes worden door touchscreen-drivers aangeroepen, zodat de code in gebruikersinterface (min of meer) hardware-neutraal is.

De methode *linkTouchScreenInputDevice* koppelt een touchscreen als invoer-apparaat aan de gebruikersinterface. Het type **TouchScreenInputDevice** is de interface van een touchscreen invoer-driver en wordt later besproken.

De concrete gebruikersinterface is geïmplementeerd in de klasse **WeatherStationUserInterface**:
```Java
public class WeatherStationUserInterface implements UserInterface {

    public WeatherStationUserInterface() {
        System.out.println("Weerstation UI versie 1.0");
    }

    @Override
    public void linkTouchScreenInputDevice(TouchScreenInputDevice touchScreenInputDevice) {
        touchScreenInputDevice.registerUserInterface(this);
    }

    @Override
    public void onTouch(int x, int y) {
        System.out.println("WEERSTATION UI scherm aangeraakt "+x+","+y);
    }

    @Override
    public void onLongTouch(int x, int y) {
        System.out.println("WEERSTATION UI scherm aangeraakt [lang] "+x+","+y);
    }

}
```

Maak de package *ui* en maak de interface **UserInterface** en klasse **WeatherStationUserInterface**.

## Touchscreen input-drivers
De drivers voor touchscreen-input bevinden zich in de package drivers.touchscreeninput.

De schermen in weerstations kunnen afkomstig zijn van verschillende fabrikanten. Voor elke fabrikant is een driver nodig om het scherm te ondersteunen.

Elke touchscreen input-driver implementeer de interface **TouchScreenInputDevice**:
```Java
public interface TouchScreenInputDevice {

    void registerUserInterface(UserInterface userInterface);

}
```

Deze interface bestaat uit slechts één methode *registerUserInterface*: Een gebruikers-interface registreren bij de driver, zodat de driver in staat is om de methoden *onTouch* en *onLongTouch* aan te roepen als de gebruiker op het scherm drukt.

Naast deze methode zal elke driver veel meer methoden bevatten, maar deze zijn driver-specifiek. De enige algemene methode is *registerUserInterface* waarmee communicatie tot stand komt tussen de verschillende touchscreen input-drivers en de gebruikersinterface.

De eerste versie van het weerstation heeft een touchscreen van het type ABCMega.

Dit is de code van de driver:
```Java
public class TouchScreenInputABCMega implements TouchScreenInputDevice {

    private UserInterface userInterface;

    @Override
    public void registerUserInterface(UserInterface userInterface) {
        this.userInterface = userInterface;
    }

    // Hardware specifieke functies (afhankelijk van merk en type)

    public void simulateUserActionPress(int x, int y) {
        if (userInterface!=null) {
            userInterface.onTouch(x,y);
        }
    }

    public void simulateUserActionLongPress(int x, int y) {
        if (userInterface!=null) {
            userInterface.onLongTouch(x,y);
        }
    }

}
```

## Het besturingssysteem starten
De volgende code in de main-methode initialiseert het besturingssysteem
```Java
WeatherStationUserInterface weatherStationUserInterface = new WeatherStationUserInterface();

// Touchscreen input device driver instantiëren en koppelen
TouchScreenInputABCMega touchScreen = new TouchScreenInputABCMega();
weatherStationUserInterface.linkTouchScreenInputDevice(touchScreen);
```

Test de code.

Er gebeurt niets, omdat er niets met het touchscreen gebeurt.
Met de methodes *simulateUserActionPress* en *simulateUserActionLongPress* kunnen acties van een gebruiker gesimuleerd worden:
```Java
// Acties van een gebruiker met het touchscreen (gesimuleerd)
touchScreen.simulateUserActionPress(100, 120);
touchScreen.simulateUserActionLongPress(40, 178);
```

Als bovenstaande code wordt uitgevoerd, dan zal het resultaat zijn:
```
Weerstation UI versie 1.0
WEERSTATION UI scherm aangeraakt 100,120
WEERSTATION UI scherm aangeraakt [lang] 40,178
```


## Muizen
Een ouder en welbekend invoer-apparaat is de muis. Bij het ontwikkelen van het weerstation is nooit rekening gehouden met muizen.

Op verzoek van een aantal gebruikers is besloten om muizen te ondersteunen. Dat is niet eenvoudig, maar een muis van het type *XYZSuperMouse* blijkt goed samen te kunnen werken met de hardware.

Voor deze muis zijn al drivers beschikbaar. De ontwikkelaars gaan bestaande drivers gebruiken.

Muizen zijn een afzonderlijke categorie apparaten. Drivers staan in package *drivers.mouse*.

Elke muis-driver implementeert te interface *Mouse*:
```Java
public interface Mouse {

    public static final int BUTTON_NONE = 0;
    public static final int BUTTON_LEFT = 1;
    public static final int BUTTON_RIGHT = 2;

    int getX();
    int getY();
    int getButtonStatus();
    void setOnAction(Consumer<Mouse> action);

}
```

Dit is de code van de driver van *XYZSuperMouse*:
```Java
public class XYZSuperMouse implements Mouse {

    private int x,y, buttonStatus;
    private Consumer<Mouse> action;

    @Override
    public int getX() {
        return x;
    }

    @Override
    public int getY() {
        return y;
    }

    @Override
    public int getButtonStatus() {
        return buttonStatus;
    }

    @Override
    public void setOnAction(Consumer<Mouse> action) {
        this.action = action;
    }

    // Hardware specifieke functies (afhankelijk van merk en type)

    private void checkForAction() {
        if (action==null) {
            throw new IllegalStateException("No action set");
        }
    }

    public void simulateUserActionMove(int x, int y) {
        checkForAction();
        this.x=x;
        this.y=y;
        action.accept(this);
    }

    public void simulateUserActionClickLeft() {
        checkForAction();
        this.buttonStatus = BUTTON_LEFT;
        action.accept(this);
        this.buttonStatus = BUTTON_NONE;
    }

    public void simulateUserActionClickRight() {
        checkForAction();
        this.buttonStatus = BUTTON_RIGHT;
        action.accept(this);
        this.buttonStatus = BUTTON_NONE;
    }


}
```

## De muis-driver
Een muis is geen touchscreen, ook al worden beide gebruikt om te 'klikken' op een bepaalde plek op het scherm. De driver van een muis steekt ook heel ander in elkaar dan de driver van touchscreen input.

Een muis koppelen aan de gebruikersinterface is niet mogelijk. Blijkbaar is de gebruikers-interface iets minder hardware-neutraal dan de ontwikkelaars dachten! Er is een sterke koppeling tussen gebruikersinterface en touchscreens input-drivers.

Op langere termijn zal dit ontwerp herzien moeten worden, maar op korte termijn willen de ontwikkelaars experimentele ondersteuning voor de *XYZSuperMouse* realiseren zonder dat de bestaande code voor de gebruikersinterface en de touchscreen-drivers aangepast hoeft te worden.

Groot verschil tussen muis en touchscreen is dat een muis coördinaten van de cursor bijhoudt, en verplaatsing van de muis zonder klikken ook een gebeurtenis is waar de driver op reageert.

Om inzicht te krijgen in het gedrag van de muis-driver, kan de volgende testcode worden uitgevoerd vanuit main:
```Java
XYZSuperMouse mouse = new XYZSuperMouse();
mouse.setOnAction(new Consumer<Mouse>() {
    @Override
    public void accept(Mouse mouse) {
        System.out.print("x:"+mouse.getX()+ " y:"+ mouse.getY()+" knop:");
        switch (mouse.getButtonStatus()) {
            case Mouse.BUTTON_LEFT:
                System.out.println("links");
                break;
            case Mouse.BUTTON_RIGHT:
                System.out.println("rechts");
                break;
            case Mouse.BUTTON_NONE:
                System.out.println("geen");
                break;
            default:
                System.out.println("onbekend");
        }
    }
});

// Acties van een gebruiker met de muis (gesimuleerd)
mouse.simulateUserActionMove(300, 150);
mouse.simulateUserActionClickLeft();
mouse.simulateUserActionMove(300, 130);
mouse.simulateUserActionMove(200, 130);
mouse.simulateUserActionClickRight();
```

Voor de *setOnAction* wordt een *anonieme klasse* gebruikt. Om het voorbeeld te gebruiken is het niet noodzakelijk om het concept van anonieme klassen te begrijpen.

Deze code is alleen bedoeld om de muisdriver te bestuderen en maakt geen vast onderdeel uit van main.

De uitvoer is:
```
x:300 y:150 knop:geen
x:300 y:150 knop:links
x:300 y:130 knop:geen
x:200 y:130 knop:geen
x:200 y:130 knop:rechts
```

## Adapter pattern
De muisdriver koppelen aan de gebruikersinterface lukt niet:
```Java
// Touchscreen mouse driver instantiëren en koppelen
XYZSuperMouse mouse = new XYZSuperMouse();
weatherStationUserInterface.linkTouchScreenInputDevice(mouse); // Compileert niet
```

Gebruik het adapter pattern om dit probleem op te lossen. De adapter gedraagt zich als touchscreen invoer-apparaat (implementeert **TouchScreenInputDevice**) en bevat een instantie van **Mouse**.

Er zijn een aantal fundamentele verschillen tussen een muis en touchscreen:
- Klikken: Een touchscreen werkt met kort of lang drukken, een muis met verschillende koppen. De linker muisknop staat gelijk aan een korte druk op het touchscreen, de rechter muisknop is een lange druk
- Muiscursor: Voor de muis is een cursor (pijltje) nodig. Ga er vanuit dat dit is geregeld in de uitvoer-drivers die buiten deze opgave vallen.

Maak de klasse **MouseAdapter**.

Hint: De klasse **MouseAdapter** moet de methode *setOnAction* van de muis aanroepen om de klikken op te vangen en door te geven aan de gebruikersinterface. Dit kan op heel veel verschillende manieren! De meest eenvoudige manier:  een klasse **MouseAdapterClick** die de interface **Consumer<Mouse>** implementeert.

## Testen
Als de adapter goed functioneert, voer dan de volgende test uit:
```Java
WeatherStationUserInterface weatherStationUserInterface = new WeatherStationUserInterface();

// Touchscreen input device driver instantiëren en koppelen
TouchScreenInputABCMega touchScreen = new TouchScreenInputABCMega();
weatherStationUserInterface.linkTouchScreenInputDevice(touchScreen);

// Touchscreen mouse driver instantiëren en koppelen
XYZSuperMouse mouse = new XYZSuperMouse();
// hier hoort een regel te staan die de muis koppelt aan de gebruikersinterface

// Acties van een gebruiker met het touchscreen (gesimuleerd)
touchScreen.simulateUserActionPress(100, 120);
touchScreen.simulateUserActionLongPress(40, 178);

// Acties van een gebruiker met de muis (gesimuleerd)
mouse.simulateUserActionMove(300, 150);
mouse.simulateUserActionClickLeft();
mouse.simulateUserActionMove(300, 130);
mouse.simulateUserActionMove(200, 130);
mouse.simulateUserActionClickRight();

```
Verwacht resultaat:
```
Weerstation UI versie 1.0
WEERSTATION UI scherm aangeraakt 100,120
WEERSTATION UI scherm aangeraakt [lang] 40,178
WEERSTATION UI scherm aangeraakt 300,150
WEERSTATION UI scherm aangeraakt [lang] 200,130
```
Uit dit resultaat blijkt dat het scherm zowel invoer van de muis als het touchscreen krijgt.

## Extra uitdaging
De sterke koppeling tussen de gebruikersinterface en de touchscreen input-drivers is ongewenst. Welke design patterns kunnen worden ingezet om de gebruikersinterface te ontkoppelen van touchscreen input-drivers?

Als de gebruikersinterface een generieke interface gebruikt voor input-drivers, zou het adapter pattern dan overbodig zijn als het gaat om het ondersteunen van de *XYZSuperMouse*?